In [1]:
import torch

In [2]:
torch.__version__

'1.5.1+cu101'

In [3]:
# 全結合層の定義
import torch.nn as nn

In [4]:
# 全結合層の定義
fc = nn.Linear(3, 2)

# (線形結合)

In [5]:
# CPU のシードを固定
torch.manual_seed(1)

In [6]:
# 重み
fc.weight

Parameter containing:
tensor([[ 0.0818, -0.5633,  0.4698],
        [ 0.1824, -0.2893,  0.1391]], requires_grad=True)

In [7]:
# バイアス
fc.bias

Parameter containing:
tensor([0.2064, 0.5493], requires_grad=True)

In [8]:
# CPU のシードを固定
torch.manual_seed(1)

In [9]:
# リストから PyTorch の Tensor に変換
x = torch.tensor([[1, 2, 3]])

type(x)

torch.Tensor

詳細なデータ型の設定が必要

In [10]:
# 詳細なデータ型の確認
x.dtype

torch.int64

In [11]:
# float32 に変換
x = torch.tensor([[1, 2, 3]], dtype=torch.float32)
# x = torch.tensor([[1, 2, 3]], dtype=torch.FloatTensor)

x, x.dtype

(tensor([[1., 2., 3.]]), torch.float32)

In [12]:
# 線形変換
u = fc(x)
u

tensor([[0.5711, 0.5703]], grad_fn=<AddmmBackward>)

In [13]:
import torch.nn.functional as F

In [14]:
# 活性化関数(activation)の導入
# ReLU 関数
z = F.relu(u)
z

tensor([[0.5711, 0.5703]], grad_fn=<ReluBackward0>)

例題

In [15]:
torch.manual_seed(1)

In [16]:
#　入力値の定義
x = torch.tensor([[1,2,3]], dtype=torch.float32)

In [17]:
# 全結合層の定義
fc1 = nn.Linear(3,2)
fc2 = nn.Linear(2,1)

In [18]:
# fc1線形変換
u1 = fc1(x)
u1

tensor([[-0.6667,  0.5164]], grad_fn=<AddmmBackward>)

In [19]:
# 活性化関数(activation)の導入
# ReLU 関数
z1 = F.relu(u1)
z1

tensor([[0.0000, 0.5164]], grad_fn=<ReluBackward0>)

In [20]:
# fc2線形変換
y = fc2(z1)
y

tensor([[0.1514]], grad_fn=<AddmmBackward>)

In [21]:
# 目標値の設定
# t=1
t = torch.tensor([[1]], dtype=torch.float32)
t

tensor([[1.]])

目的関数

torch.nn.function(F)で定義
1. torch.nn (nn): 調整すべきパラメータを持つ
2. torch.nn.functional (F) : 調整すべきパラメータを持たない

In [22]:
# 平均二乗誤差
loss = F.mse_loss(t, y)
loss

tensor(0.7201, grad_fn=<MeanBackward0>)

In [23]:
from sklearn.datasets import load_iris

# Iris データセットの読み込み
x, t = load_iris(return_X_y=True)

In [24]:
# 形の確認
x.shape, t.shape

((150, 4), (150,))

In [25]:
# 型の確認
type(x), type(t)

(numpy.ndarray, numpy.ndarray)

In [26]:
# データ型の確認
x.dtype, t.dtype

(dtype('float64'), dtype('int64'))

In [27]:
# わかりやすくするため、一応再度
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [28]:
# データ型の変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [29]:
# 型の確認
type(x), type(t)

(torch.Tensor, torch.Tensor)

In [30]:
# データ型の確認
x.dtype, t.dtype

(torch.float32, torch.int64)

In [31]:
# 入力変数と目的変数をまとめて、ひとつのオブジェクト dataset に変換
dataset = torch.utils.data.TensorDataset(x, t)
dataset

In [32]:
# 型の確認
type(dataset)

torch.utils.data.dataset.TensorDataset

In [33]:
# (入力値, 目標値) のようにタプルで格納されている
dataset[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor(0))

In [34]:
# 型の確認
type(dataset[0])

tuple

In [35]:
# 1 サンプル目の入力値
dataset[0][0]

tensor([5.1000, 3.5000, 1.4000, 0.2000])

In [36]:
# 1 サンプル目の目標値
dataset[0][1]

tensor(0)

In [37]:
# サンプル数は len で取得可能
len(dataset)

150

In [38]:
# 各データセットのサンプル数を決定
# train : val: test = 60%　: 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

In [39]:
# それぞれのサンプル数を確認
n_train, n_val, n_test

(90, 30, 30)

In [40]:
# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [41]:
# サンプル数の確認
len(train), len(val), len(test)

(90, 30, 30)

# ミニバッチ学習
いくつかのデータをまとめて入力し、それぞれの勾配を計算したあと、その勾配の平均値を用いてパラメータの更新を行う方法

In [42]:
# バッチサイズ
batch_size = 10

In [43]:
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

順伝播の計算の流れ
1. 線形変換 (fc1)
2. 非線形変換 (ReLU)
3. 線形変換 (fc2)
4. 非線形変換 (Softmax)

In [44]:
class Net(nn.Module):

    # 使用するオブジェクトを定義
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3)

    # 順伝播
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [45]:
# 乱数のシードを固定して再現性を確保
torch.manual_seed(0)

# インスタンス化
net = Net()

In [46]:
# モデルの確認
net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

目的関数:クロスエントロピー

PyTorchでは計算を高速化させるために、Softmax 関数と対数変換を同時に行う関数であるF.log_softmax が用意されている。

PyTorch で用意されているF.cross_entropyでは内部の計算に F.log_softmax が使用されている。

PyTorch では目的関数を criterion という名前で定義することが一般的。

In [47]:
# 目的関数の設定
criterion = F.cross_entropy
criterion

<function torch.nn.functional.cross_entropy>

最適化手法の選択
- optimizer という変数名で定義
- optimizer を定義する際に、引数としてモデルのパラメータを渡す必要があり、パラメータの取得には net.parameters()を使用
- もう１つの引数として学習係数 (lr: learning rate) も選択

In [48]:
# net.parameters() を展開
for parameter in iter(net.parameters()):
    print(parameter)

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)
Parameter containing:
tensor([ 0.1977,  0.3000, -0.3390, -0.2177], requires_grad=True)
Parameter containing:
tensor([[ 0.1816,  0.4152, -0.1029,  0.3742],
        [-0.0806,  0.0529,  0.4527, -0.4638],
        [-0.3148, -0.1266, -0.1949,  0.4320]], requires_grad=True)
Parameter containing:
tensor([-0.3241, -0.2302, -0.3493], requires_grad=True)


In [49]:
# 最適化手法の選択
# lr(学習係数)
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)

# モデルの学習手順
1. ミニバッチ単位でサンプル X,t を抽出
2. 現在のパラメータ W,b を利用して、順伝播で予測値yを算出
3. 目標値tと予測値yから目的関数Lを算出
4. 誤差逆伝播法に基づいて各パラメータの勾配を算出
5. 勾配の値に基づいて選択した最適化手法によりパラメータ W,b を更新

In [50]:
# バッチサイズ分のサンプルの抽出
batch = next(iter(train_loader))
batch

[tensor([[6.4000, 3.2000, 5.3000, 2.3000],
         [6.4000, 2.7000, 5.3000, 1.9000],
         [5.9000, 3.0000, 4.2000, 1.5000],
         [6.9000, 3.1000, 5.4000, 2.1000],
         [5.5000, 2.4000, 3.8000, 1.1000],
         [5.6000, 2.7000, 4.2000, 1.3000],
         [5.8000, 2.7000, 5.1000, 1.9000],
         [6.9000, 3.1000, 4.9000, 1.5000],
         [6.3000, 2.3000, 4.4000, 1.3000],
         [7.2000, 3.0000, 5.8000, 1.6000]]),
 tensor([2, 2, 1, 2, 1, 1, 2, 1, 1, 2])]

In [51]:
# 入力値と目標値に分割
x, t = batch

In [52]:
# 入力値の確認
x

tensor([[6.4000, 3.2000, 5.3000, 2.3000],
        [6.4000, 2.7000, 5.3000, 1.9000],
        [5.9000, 3.0000, 4.2000, 1.5000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [5.5000, 2.4000, 3.8000, 1.1000],
        [5.6000, 2.7000, 4.2000, 1.3000],
        [5.8000, 2.7000, 5.1000, 1.9000],
        [6.9000, 3.1000, 4.9000, 1.5000],
        [6.3000, 2.3000, 4.4000, 1.3000],
        [7.2000, 3.0000, 5.8000, 1.6000]])

In [53]:
# 目標値の確認
t

tensor([2, 2, 1, 2, 1, 1, 2, 1, 1, 2])

In [54]:
# 現状確認
# 全結合層 fc1 の重み
net.fc1.weight

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)

In [55]:
# 全結合層 fc1 のバイアス
net.fc1.bias

Parameter containing:
tensor([ 0.1977,  0.3000, -0.3390, -0.2177], requires_grad=True)

In [56]:
# 全結合層 fc2 の重み
net.fc2.weight

Parameter containing:
tensor([[ 0.1816,  0.4152, -0.1029,  0.3742],
        [-0.0806,  0.0529,  0.4527, -0.4638],
        [-0.3148, -0.1266, -0.1949,  0.4320]], requires_grad=True)

In [57]:
# 全結合層 fc2 のバイアス
net.fc2.bias

Parameter containing:
tensor([-0.3241, -0.2302, -0.3493], requires_grad=True)

In [58]:
# 予測値の算出
y = net.forward(x)
y

tensor([[-0.1763, -0.2113, -0.3944],
        [-0.2700, -0.2233, -0.3658],
        [-0.2746, -0.2239, -0.3644],
        [-0.2552, -0.2214, -0.3703],
        [-0.3241, -0.2302, -0.3493],
        [-0.3003, -0.2271, -0.3566],
        [-0.2212, -0.2171, -0.3807],
        [-0.3241, -0.2302, -0.3493],
        [-0.3241, -0.2302, -0.3493],
        [-0.3241, -0.2302, -0.3493]], grad_fn=<AddmmBackward>)

In [59]:
# call メソッドを用いた forward の計算（推奨）
y = net(x)
y

tensor([[-0.1763, -0.2113, -0.3944],
        [-0.2700, -0.2233, -0.3658],
        [-0.2746, -0.2239, -0.3644],
        [-0.2552, -0.2214, -0.3703],
        [-0.3241, -0.2302, -0.3493],
        [-0.3003, -0.2271, -0.3566],
        [-0.2212, -0.2171, -0.3807],
        [-0.3241, -0.2302, -0.3493],
        [-0.3241, -0.2302, -0.3493],
        [-0.3241, -0.2302, -0.3493]], grad_fn=<AddmmBackward>)

In [60]:
# 目的関数の計算 criterion の call メソッドを利用
loss = criterion(y, t)
loss

tensor(1.1118, grad_fn=<NllLossBackward>)

In [61]:
# 勾配の算出
loss.backward()

In [62]:
# 全結合層 fc1 の重みに関する勾配
net.fc1.weight.grad

tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.7165, 0.3319, 0.5857, 0.2248],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])

In [63]:
# 全結合層 fc1 のバイアスに関する勾配
net.fc1.bias.grad

tensor([0.0000, 0.1137, 0.0000, 0.0000])

In [64]:
# 全結合層 fc2 の重みに関する勾配
net.fc2.weight.grad

tensor([[ 0.0000,  0.0375,  0.0000,  0.0000],
        [ 0.0000,  0.0202,  0.0000,  0.0000],
        [ 0.0000, -0.0577,  0.0000,  0.0000]])

In [65]:
# 全結合層 fc2 のバイアスに関する勾配
net.fc2.bias.grad

tensor([ 0.3361, -0.1451, -0.1909])

In [66]:
# 勾配の情報を用いたパラメータの更新
optimizer.step()

In [67]:
# 全結合層 fc1 の重み
net.fc1.weight

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.2642,  0.1009, -0.0685,  0.3740],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)

In [68]:
# 全結合層 fc1 のバイアス
net.fc1.bias

Parameter containing:
tensor([ 0.1977,  0.2886, -0.3390, -0.2177], requires_grad=True)

In [69]:
# 全結合層 fc2 の重み
net.fc2.weight

Parameter containing:
tensor([[ 0.1816,  0.4114, -0.1029,  0.3742],
        [-0.0806,  0.0509,  0.4527, -0.4638],
        [-0.3148, -0.1208, -0.1949,  0.4320]], requires_grad=True)

In [70]:
# 全結合層 fc2 のバイアス
net.fc2.bias

Parameter containing:
tensor([-0.3577, -0.2157, -0.3302], requires_grad=True)

In [71]:
# 演算に使用できる GPU の有無を確認
torch.cuda.is_available()

True

In [72]:
# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [73]:
# 指定したデバイスへのモデルの転送
net.to(device)

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [74]:
# 指定したデバイスへの入力変数の転送
x = x.to(device)
x

tensor([[6.4000, 3.2000, 5.3000, 2.3000],
        [6.4000, 2.7000, 5.3000, 1.9000],
        [5.9000, 3.0000, 4.2000, 1.5000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [5.5000, 2.4000, 3.8000, 1.1000],
        [5.6000, 2.7000, 4.2000, 1.3000],
        [5.8000, 2.7000, 5.1000, 1.9000],
        [6.9000, 3.1000, 4.9000, 1.5000],
        [6.3000, 2.3000, 4.4000, 1.3000],
        [7.2000, 3.0000, 5.8000, 1.6000]], device='cuda:0')

In [75]:
# 指定したデバイスへの目的変数の転送
t = t.to(device)
t

tensor([2, 2, 1, 2, 1, 1, 2, 1, 1, 2], device='cuda:0')

In [76]:
# 勾配情報の初期化
optimizer.zero_grad()

In [77]:
# 初期化後の勾配情報
net.fc1.weight.grad

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], device='cuda:0')

In [78]:
net.fc1.bias.grad

tensor([0., 0., 0., 0.], device='cuda:0')

In [115]:
# エポックの数
max_epoch = 1

      
# モデルの初期化
torch.manual_seed(0)

# モデルのインスタンス化とデバイスへの転送
net = Net().to(device)

      
# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

      
# 学習ループ
for epoch in range(max_epoch):
    
    for batch in train_loader:
        
        # バッチサイズ分のサンプルを抽出
        x, t = batch
        
        # 学習時に使用するデバイスへデータの転送
        x = x.to(device)
        t = t.to(device)
        
        # パラメータの勾配を初期化
        optimizer.zero_grad()
        
        # 予測値の算出
        y = net(x)
        
        # 目標値と予測値から目的関数の値を算出
        loss = criterion(y, t)
        
        # 目的関数の値を表示して確認
        # item(): tensot.Tensor => float
        print('loss: ', loss.item())
        
        # 各パラメータの勾配を算出
        loss.backward()
        
        # 勾配の情報を用いたパラメータの更新
        optimizer.step()

loss:  1.1118199825286865
loss:  1.0638254880905151
loss:  1.0752707719802856
loss:  1.0017006397247314
loss:  1.0963518619537354
loss:  0.9922469258308411
loss:  0.9230359792709351
loss:  0.8453548550605774
loss:  0.8222911953926086


In [116]:
# dim=1 で行ごとの最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y, dim=1)

In [117]:
# 予測値から最大となるクラスの番号を取り出した結果
y_label

tensor([0, 0, 1, 0, 0, 1, 1, 0, 0, 0], device='cuda:0')

In [118]:
# 値が一致しているか確認
y_label == t

tensor([ True, False, False,  True,  True, False, False, False,  True,  True],
       device='cuda:0')

In [119]:
# 値が True となる個数の総和
torch.sum(y_label == t)

tensor(5, device='cuda:0')

In [120]:
# int => float 
torch.sum(y_label == t) * 1.0

tensor(5., device='cuda:0')

In [121]:
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc

tensor(0.5000, device='cuda:0')

In [122]:
# モデルの初期化
torch.manual_seed(0)

# モデルのインスタンス化とデバイスへの転送
net = Net().to(device)

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [123]:
for epoch in range(max_epoch):

    for batch in train_loader:

        x, t = batch
        x = x.to(device)
        t = t.to(device)
        optimizer.zero_grad()
        y = net(x)
        loss = criterion(y, t)

        # New：正解率の算出
        y_label = torch.argmax(y, dim=1)
        acc  = torch.sum(y_label == t) * 1.0 / len(t)
        print('accuracy:', acc)

        loss.backward()
        optimizer.step()

accuracy: tensor(0.5000, device='cuda:0')
accuracy: tensor(0.5000, device='cuda:0')
accuracy: tensor(0.3000, device='cuda:0')
accuracy: tensor(0.9000, device='cuda:0')
accuracy: tensor(0.1000, device='cuda:0')
accuracy: tensor(0.6000, device='cuda:0')
accuracy: tensor(0.9000, device='cuda:0')
accuracy: tensor(0.5000, device='cuda:0')
accuracy: tensor(0.5000, device='cuda:0')


In [124]:
# 検証データやテストデータに対しては学習を行わないため、勾配の情報が必要ない

# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            x = x.to(device)
            t = t.to(device)
            y = net(x)
            
            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc

In [125]:
# 検証データで確認
calc_acc(val_loader)

Accuracy: 60.0%


tensor(0.6000)

In [126]:
# テストデータで確認
calc_acc(test_loader)

Accuracy: 43.3%


tensor(0.4333)